In [ ]:
#Latency of image arrival at USDF after shutter close on LSSTCam

In [ ]:
# This cell is only for setting parameter defaults
day = "20240925"

In [ ]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry
import sqlalchemy
from astropy import time


repo = 'embargo_new'
instrument = 'LSSTCam'
# raw all collection contains all images 
collections = [f'{instrument}/raw/all']
butler = Butler(repo, collections=collections, instrument=instrument)
where = f"day_obs={day}"

# Data transferred and ingested in butler and access to data



Get the ingest time for each image in the day

In [ ]:

with butler.query() as q:
    q = q.join_dataset_search(dt,collections)
    q = q.where(day_obs=int(day))
    result = list(q.general(["exposure"], "raw.ingest_date", find_first=False))

print (f"Got ingest times for {len(result)} images")


In [ ]:
# Make a map of detectors per exposure for look up below .. its faster than looking up the exposure time for each
# Shutter close is the same for all detector images in the same exposure - so basically grouping. 
dslookup = {}
for ref in result:
   if ref['exposure'] in dslookup:
       exp = dslookup[ref['exposure']] 
   else:  
       exp = {}
       dslookup[ref['exposure']] = exp
   exp[ref['detector']] = ref['raw.ingest_date']

print(f"The images map to {len(dslookup)} exposures.")

### latency ..
"Measure the delays between completion of readout for each image and the ingestion time recorded in the Butler Registry."
Get the shuter close time for each exposre 
the ID in the dimension record is the exposure ID (dimensionRecord.id)
thed timespan.end is the sutter close time 


In [ ]:
shuterTimes = butler.registry.queryDimensionRecords('exposure', where=where, instrument=instrument)

print (f"Got Metadata (DimensionRecords) for {res.count()} exposures")
times = []
min = 100
max = 0
for count, dimensionRecord in enumerate(shuterTimes):
    detectorMap = dslookup[dimensionRecord.id]
    
    for dcount, detector in enumerate(detectorMap): 
        intime = detectorMap[detector]
        # just do some for now too long for noteburst
        diff = (intime - dimensionRecord.timespan.end ).to('minute')
        if diff.value < min:
            min = diff.value
        if diff.value > max:
            max = diff.value 
        times.append(diff.value)
       
    
import numpy as np
print (f"The min time was {min} max time was {max} minutes over {len(times)} times.")
print (f"Time between shutter close and ingest mean:{np.mean(times)}, std:{np.std(times)}, var:{np.var(times)} minutes")

In [ ]:
import matplotlib.pyplot as plt
seconds = list(map(lambda t:t*60.0,times))
plt.hist(seconds,bins='auto')
plt.title(f"Shutter close to ingest times(s) for {len(times)} images on {day}")
plt.show()